# Imports and setting up viz

NB : conda env1 on PC, lam1env on spirit (Python3.12)

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

#import personnal tools
from tools import *

# Load and edit files

LAM output analysis.

simu irr et simu no-irr.

## Sims

In [ ]:
noirr_dir='../../../JZ_simu_outputs/LAM/noirr_2010_2022'
irr_dir='../../../JZ_simu_outputs/LAM/irr_2010_2022'

In [ ]:
# #open netcdf files for native routing on orc grid
# filename = '{}/SRF/MO/*history.nc'.format(noirr_dir)
# orc0 = xr.open_mfdataset(filename)
# #name
# orc0.attrs['name'] = 'no_irr'
# #time counter
# orc = orc0.rename({'time_counter':'time'})
# #define vars
# orc['totrunoff'] = orc['runoff'] + orc['drainage']
# # limit data to lon and lat of interest (-13,6) and (32,49)
# # orc = orc.sel(lon=slice(-13,6),lat=slice(32,49))
# orc


In [ ]:
# filename = '{}/SRF/MO/*history.nc'.format(irr_dir)
# orc0irr = xr.open_mfdataset(filename)
# #name
# orc0irr.attrs['name'] = 'irr'
# #time counter
# orcirr = orc0irr.rename({'time_counter':'time'})
# #define vars
# orcirr['totrunoff'] = orcirr['runoff'] + orcirr['drainage']
# # limit data to lon and lat of interest (-13,6) and (32,49)
# # orc = orc.sel(lon=slice(-13,6),lat=slice(32,49))
# orcirr

In [ ]:
#sim
filename = '{}/ATM/TS*.nc'.format(noirr_dir)
filename = '{}/ATM/MO/*.nc'.format(noirr_dir)

sim0 = xr.open_mfdataset(filename)
sim0.attrs['name'] = 'no_irr'
sim = sim0.rename({'time_counter':'time'})
sim = sim.sel(lon=slice(-13,6),lat=slice(32,49))

sim['evap'] = sim['evap'] *3600 * 24
sim['evap'].attrs['units'] = 'mm/d'

sim['precip'] = sim['precip'] *3600 * 24
sim['precip'].attrs['units'] = 'mm/d'

sim['fluxsens']= -sim['sens']

sim['netrad'] = sim['LWdnSFC'] - sim['LWupSFC'] + sim['SWdnSFC'] - sim['SWupSFC']
sim['netrad'].attrs['units'] = 'W/m2'

sim['P - E'] = sim['precip'] - sim['evap']
sim['P - E'].attrs['units'] = 'mm/d'

sim

In [ ]:
#sim irr
filename = '{}/ATM/TS*.nc'.format(irr_dir)
filename = '{}/ATM/MO/*.nc'.format(irr_dir)

sim0irr = xr.open_mfdataset(filename)
sim0irr.attrs['name'] = 'irr'
simirr = sim0irr.rename({'time_counter':'time'})
simirr = simirr.sel(lon=slice(-13,6),lat=slice(32,49))

simirr['evap'] = simirr['evap'] *3600 * 24
simirr['evap'].attrs['units'] = 'mm/d'

simirr['precip'] = simirr['precip'] *3600 * 24
simirr['precip'].attrs['units'] = 'mm/d'

simirr['fluxsens']= -simirr['sens']

simirr['netrad'] = simirr['LWdnSFC'] - simirr['LWupSFC'] + simirr['SWdnSFC'] - simirr['SWupSFC']
simirr['netrad'].attrs['units'] = 'W/m2'

simirr['P - E'] = simirr['precip'] - simirr['evap']
simirr['P - E'].attrs['units'] = 'mm/d'

simirr


## Obs

In [ ]:
#open ERA5 file
filename = '../../../obs/ERA5_eval_2010_2022.nc'
era = xr.open_mfdataset(filename)
era.attrs['name'] = 'ERA5'
#define or edit vars
#name ERA5 vars like in LMDZOR outputs
dict =  {
        'longitude':'lon',
        'latitude':'lat',
        'e':'evap',
        'ro':'totrunoff',
        'tp':'precip'
        }
era = era.rename(dict)

#make evap positive in era
era['evap'] = -era['evap'] * 1000
era['evap'].attrs['units'] = 'mm/d'
era['precip'] = era['precip']*1000
era['precip'].attrs['units'] = 'mm/d'
era['cldt'] = era['tcc']*100
era['cldt'].attrs['units'] = '%'

era['P - E'] = era['precip'] - era['evap']
era['P - E'].attrs['units'] = 'mm/d'

era


In [ ]:
#gleam
filename='../../../obs/evap/E_1980-2022_GLEAM_v3.8a_MO.nc'
gleam0=xr.open_dataset(filename)
gleam0.attrs['name']='GLEAM'
gleam = gleam0.sel(lon=slice(-13,6),lat=slice(49,32))
#adapt period
gleam=gleam.sel(time=slice('2010-01-01', '2022-12-31'))
#name gleam vars like in LMDZOR outputs
dict =  {
        'E':'evap'
        }
gleam = gleam.rename(dict)
#convert mm/month to mm/day
gleam['evap']=gleam['evap']/30
gleam['evap'].attrs['units']='mm/d'
gleam

In [ ]:
filename='../../..//obs/FluxCom/ensemble_rs-_meteo_ALL/monthly/*.RS_METEO.EBC-*.MLM-ALL.METEO-ALL.720_360.monthly.*.nc'
fluxcom0=xr.open_mfdataset(filename)
fluxcom=fluxcom0.sel(lon=slice(-13,6.25),lat=slice(49,32))
fluxcom=fluxcom.sel(time=slice('2010-01-01', '2022-12-31'))

fluxcom.attrs["name"]='FluxCom'

fluxcom['evap'] = fluxcom['LE'] * 0.408 # To convert from MJ/m2/d to mm/d
fluxcom['evap'].attrs['units'] = 'mm/d'

fluxcom['fluxlat'] = fluxcom['LE'] / 0.0864 # To convert from MJ/m2/d to W/m2
fluxcom['fluxlat'].attrs['units'] = 'W/m²'

fluxcom['fluxsens'] = fluxcom['H'] / 0.0864 # To convert from MJ/m2/d to W/m2
fluxcom['fluxsens'].attrs['units'] = 'W/m²'

fluxcom['netrad'] = fluxcom['Rn'] / 0.0864 # To convert from MJ/m2/d to W/m2
fluxcom['netrad'].attrs['units'] = 'W/m²'
fluxcom

In [ ]:
#gpcc
filename='../../../obs/precips/precip.mon.total.0.25x0.25.v2020.nc'
gpcc0=xr.open_mfdataset(filename)
gpcc0.attrs['name'] = 'GPCC'
gpcc=gpcc0.sel(time=slice('2010-01-01', '2022-12-31'))
gpcc['lon'] = ((gpcc['lon'] + 180) % 360) - 180
gpcc1 = gpcc.sortby('lon')

# gpcc=gpcc.sel(lon=slice(-13,6.25),lat=slice(32,49))
gpcc=gpcc1.sel(lon=slice(-13,6.25),lat=slice(49,32))

# gpcc1=gpcc.sel(lon=slice(347,360),lat=slice(49,32))
# gpcc2=gpcc.sel(lon=slice(0,6.25),lat=slice(49,32))
# gpcc=xr.combine_by_coords([gpcc1, gpcc2])
# gpcc = gpcc.where(gpcc['lon'] < 6.25, drop=True)

# gpcc['lon']=gpcc['lon']-180.0
# gpcc=gpcc.sel(lon=slice(-13,6.25))

gpcc['precip'] = gpcc['precip'] / 30 #convert to mm/d
gpcc['precip'].attrs['units'] = 'mm/d'
gpcc

In [ ]:
#open tqruv file
filename='../../../obs/TQRUV_850_2010_2022_monthly.nc'
tqruv = xr.open_dataset(filename)
tqruv.attrs['name'] = 'ERA5'
tqruv = tqruv.sel(longitude=slice(-13,6),latitude=slice(49,32))
#rename longitude and latitude to lon and lat
tqruv = tqruv.rename({'longitude':'lon','latitude':'lat'})

#rename all variables to add 850 in name
tqruv['u850'] = tqruv['u']
tqruv['v850'] = tqruv['v']
tqruv['q850'] = tqruv['q']
tqruv['t850'] = tqruv['t']
tqruv['r850'] = tqruv['r']
tqruv

# Masking, interpolation, and time period selection

In [ ]:
# Select months
# months=[7]

# orc = orc.sel(time=orc['time.month'].isin(months))
# orcirr = orcirr.sel(time=orcirr['time.month'].isin(months))

# sim = sim.sel(time=sim['time.month'].isin(months))
# simirr = simirr.sel(time=simirr['time.month'].isin(months))

# era = era.sel(time=era['time.month'].isin(months))
# fluxcom=fluxcom.sel(time=fluxcom['time.month'].isin(months))

In [ ]:
#continental fraction mask (LMDZ outputs)
con_mask=sim['contfracATM']>0.9

#create a masks of data points where irrig_frac >5%
# irr_mask = (orcirr['irrigmap_dyn']/orcirr['Areas'])>0.05
irr_mask = con_mask

ip_mask=polygon_to_mask(sim, iberic_peninsula)
# ip_mask_orc=polygon_to_mask(orc, iberic_peninsula)

In [ ]:
cont_sim=sim.where(con_mask)
cont_simirr=simirr.where(con_mask)

ip_sim = sim.where(ip_mask['mask'], drop=False).where(con_mask)
ip_simirr = simirr.where(ip_mask['mask'], drop=False).where(con_mask)

# ip_orc=orc.where(ip_mask_orc['mask'], drop=False)

In [ ]:
sim_era = sim.interp_like(era)
sim_gpcc = sim.interp_like(gpcc)
sim_gleam = sim.interp_like(gleam)
sim_fluxcom = sim.interp_like(fluxcom)
sim_tqruv = sim.interp_like(tqruv)

simirr_era = simirr.interp_like(era)
simirr_gpcc = simirr.interp_like(gpcc)
simirr_gleam = simirr.interp_like(gleam)
simirr_fluxcom = simirr.interp_like(fluxcom)
simirr_tqruv = simirr.interp_like(tqruv)

ip_sim_era = ip_sim.interp_like(era)
ip_sim_gpcc = ip_sim.interp_like(gpcc)
ip_sim_gleam = ip_sim.interp_like(gleam)
ip_sim_fluxcom = ip_sim.interp_like(fluxcom)
ip_sim_tqruv = ip_sim.interp_like(tqruv)

ip_simirr_era = ip_simirr.interp_like(era)
ip_simirr_gpcc = ip_simirr.interp_like(gpcc)
ip_simirr_gleam = ip_simirr.interp_like(gleam)
ip_simirr_fluxcom = ip_simirr.interp_like(fluxcom)
ip_simirr_tqruv = ip_simirr.interp_like(tqruv)

In [ ]:
# irrorc=orc.where(irr_mask)
# irrorcirr=orcirr.where(irr_mask)

In [ ]:
# era
int_era=era.interp_like(sim)
cont_era=int_era.where(con_mask)
ip_era = cont_era.where(ip_mask['mask'], drop=False)
# orc_era=era.interp_like(orc)

In [ ]:
#gleam
int_gleam=gleam.interp_like(sim)
cont_gleam=gleam.interp_like(sim).where(con_mask)
ip_gleam = cont_gleam.where(ip_mask['mask'], drop=False)
# irr_era=era.interp_like(orc).where(irr_mask)
# irr_gleam=gleam.interp_like(orc).where(irr_mask)

In [ ]:
#fluxcom
int_fluxcom=fluxcom.interp_like(sim)
cont_fluxcom=fluxcom.interp_like(sim).where(con_mask)
ip_fluxcom = cont_fluxcom.where(ip_mask['mask'], drop=False)
# orc_fluxcom=fluxcom.interp_like(orc)

In [ ]:
#gpcc
int_gpcc=gpcc.interp_like(sim)
cont_gpcc=gpcc.interp_like(sim).where(con_mask)
ip_gpcc = cont_gpcc.where(ip_mask['mask'], drop=False)

In [ ]:
int_tqruv = tqruv.interp_like(sim)
cont_tqruv = tqruv.interp_like(sim).where(con_mask)
ip_tqruv = cont_tqruv.where(ip_mask['mask'], drop=False)

# Maps

In [ ]:
from matplotlib.colors import LinearSegmentedColormap

# Define the custom colormap

cmap = LinearSegmentedColormap.from_list(
    "custom_green_red",
    ["#013220", "white",  "#550000"]  # Very dark green, white, dark red, almost black
)
ferret_eq=ListedColormap(cmap(np.linspace(0, 1, 16)))
ferret_eq

In [ ]:
#Display map of var for ds
var='P - E'
ds=sim_era
#select only year 2014
ds = ds.sel(time=ds['time.year'] == 2014)
color_map=ferret_eq
max_value=8

map_ave(ds, var, in_cmap=color_map, in_vmax=max_value, hex=True)

In [ ]:
#Diff map
var='q850'
max_value=7E-4
months=[1,2,3,4,5,6,7,8,9,10,11,12]
ds1=ip_sim_tqruv
ds2=tqruv
ds1=ds1.sel(time=ds1['time.month'].isin(months))
ds2=ds2.sel(time=ds2['time.month'].isin(months))

map_diff_ave(ds1, ds2, var, in_vmax=max_value, hex=True, sig=False)

diff=ds1[var]-ds2[var]
title='{} bias, {} vs {} ({})'.format( var, ds1.attrs['name'], ds2.attrs['name'], ds1[var].attrs['units'])
map_seasons(diff, in_cmap=emb, in_vmax=max_value, in_title=title, hex=True)

In [ ]:
#map for 4 seasons
var='q850'
ds1=ip_sim_tqruv
ds2=tqruv
max_value=7E-4
min_value=None
# min_value=-max_value

diff=ds1[var]-ds2[var]
title='{} bias, {} vs {} ({})'.format( var, ds1.attrs['name'], ds2.attrs['name'], ds1[var].attrs['units'])
plotvar=diff

# plotvar=ds1[var]
# title='P-E (mm/d, {})'.format(ds1.attrs['name'])

map_seasons(plotvar, in_cmap=emb, in_vmax=max_value, in_title=title, hex=True)

In [ ]:
# Display 2 maps of var
var = 'P - E'
min_value=None
max_value=None
color_map=emb_neutral
ds1=sim
ds2=simirr
map_two_ds(ds1, ds2, var, in_vmin=min_value, in_figsize=(15,6), in_vmax=max_value, in_cmap=color_map)

In [ ]:
#Display map of var for ds on restricted area
var='evap'
ds = sim
latmin=36.7
latmax=36.9
lonmin=-6.4
lonmax=-6.2

#restrict ds to latmin, latmax, lonmin, lonmax
ds = ds.where(ds.lat >= latmin, drop=True).where(ds.lat <= latmax, drop=True)
ds = ds.where(ds.lon >= lonmin, drop=True).where(ds.lon <= lonmax, drop=True)
plotvar = ds[var].mean(dim='time')
map_plotvar(plotvar, in_cmap=emb_neutral, in_vmax=None, in_title=var)

In [ ]:
#Relative diff map
var='precip'
max_value=None
ds1=cont_fluxcom
ds2=ip_fluxcom
# plt.title('Gross primary production relative difference (%)')
map_rel_diff_ave(ds1, ds2, var, in_vmax=max_value)

# Time series

In [ ]:
var='q850'

ds1=ip_sim
ds2=ip_simirr
ds3=ip_era
ds3=ip_fluxcom
ds5=ip_gleam

# ds1=ip_orc
# ds2=ip_orcirr
# ds3=ip_fluxcom_orc


ds3=ip_tqruv

ds_list=[ds1, ds2, ds3]#, ds4, ds5]
# ds_list=[ds2]

year_max=2022
time_series(ds_list, var, year_max=year_max, in_title='{} {}'.format(var, ds1[var].attrs['units']))
seasonal_cycle(ds_list, var, year_max=year_max, in_title='Seasonal cycle of {} ({})'.format(var, ds1[var].attrs['units']))